# ML Model Factory - Complete Pipeline & Training

This notebook runs the **complete ML pipeline** from raw data to trained models.

## What This Notebook Does
1. **Setup** - Mount Drive (for results), clone GitHub repo (for code & data)
2. **Phase 1** - Data pipeline (clean -> features -> labels -> splits)
3. **Phase 2** - Model training (single or multiple models)
4. **Phase 3** - Cross-validation (optional)
5. **Phase 4** - Ensemble training (optional)

## Data Flow
- **Data Source:** `/content/research/` (cloned from GitHub)
- **Results Saved:** `/content/drive/MyDrive/research/` (Google Drive for persistence)

## Quick Start
1. Run cells in order (or use Runtime -> Run all)
2. Data is loaded from the GitHub clone
3. Results are saved to Google Drive for persistence

---

## 1. Environment Setup

In [ ]:
#@title 1.1 Mount Google Drive & Clone Repository { display-mode: "form" }
#@markdown Run this cell to mount your Google Drive and set up the project.

import os
import sys
from pathlib import Path

# Mount Google Drive (for saving results only)
from google.colab import drive
drive.mount('/content/drive')

# Clone or pull repository
if not Path('/content/research').exists():
    print("Cloning repository...")
    !git clone https://github.com/Snehpatel101/research.git /content/research
else:
    print("Pulling latest changes...")
    !cd /content/research && git pull

# Change to project directory
os.chdir('/content/research')

# Create Drive directories for saving results
for d in ["experiments/runs", "results"]:
    Path('/content/drive/MyDrive/research', d).mkdir(parents=True, exist_ok=True)

print("\n" + "=" * 60)
print(" PATH CONFIGURATION")
print("=" * 60)
print(f"\nProject directory: {os.getcwd()}")
print(f"Data source: /content/research (from GitHub)")
print(f"Results saved to: /content/drive/MyDrive/research (Google Drive)")
print("=" * 60)

In [ ]:
#@title 1.2 Install Dependencies { display-mode: "form" }
#@markdown Installs all required packages for the ML pipeline.

import sys

# Add project to Python path
sys.path.insert(0, '/content/research')

# Install required packages
!pip install xgboost lightgbm catboost optuna ta pywavelets scikit-learn pandas numpy -q

# Verify PyTorch with CUDA
import torch
if torch.cuda.is_available():
    print(f"PyTorch: {torch.__version__} with CUDA {torch.version.cuda}")
else:
    print(f"PyTorch: {torch.__version__} (CPU only)")

print(f"\nProject path added: /content/research")
print("Dependencies installed!")

In [ ]:
#@title 1.3 Detect Hardware & Configure { display-mode: "form" }
#@markdown Detects GPU and configures optimal settings.

import torch
import platform

print("=" * 60)
print(" HARDWARE DETECTION")
print("=" * 60)

# System info
print(f"\nSystem: {platform.system()} {platform.release()}")
print(f"Python: {sys.version.split()[0]}")

# GPU detection
GPU_AVAILABLE = torch.cuda.is_available()
GPU_NAME = None
GPU_MEMORY = 0
RECOMMENDED_BATCH_SIZE = 256
MIXED_PRECISION = False

if GPU_AVAILABLE:
    props = torch.cuda.get_device_properties(0)
    GPU_NAME = props.name
    GPU_MEMORY = props.total_memory / (1024**3)
    
    print(f"\nGPU: {GPU_NAME}")
    print(f"Memory: {GPU_MEMORY:.1f} GB")
    print(f"Compute Capability: {props.major}.{props.minor}")
    
    if GPU_MEMORY >= 40:  # A100
        RECOMMENDED_BATCH_SIZE = 1024
        MIXED_PRECISION = True
    elif GPU_MEMORY >= 15:  # T4/V100
        RECOMMENDED_BATCH_SIZE = 512
        MIXED_PRECISION = True
    else:
        RECOMMENDED_BATCH_SIZE = 256
        MIXED_PRECISION = props.major >= 7
    
    print(f"\nRecommended batch size: {RECOMMENDED_BATCH_SIZE}")
    print(f"Mixed precision: {'Enabled' if MIXED_PRECISION else 'Disabled'}")
else:
    print("\nNo GPU detected - will use CPU")
    print("Tip: Runtime -> Change runtime type -> GPU")

# Verify model registry
print("\n" + "=" * 60)
print(" AVAILABLE MODELS")
print("=" * 60)

try:
    from src.models import ModelRegistry
    models = ModelRegistry.list_models()
    for family, model_list in models.items():
        print(f"\n{family.upper()}:")
        for m in model_list:
            gpu_req = "GPU" if m in ['lstm', 'gru', 'tcn'] else "CPU"
            print(f"  - {m} ({gpu_req})")
except Exception as e:
    print(f"Error loading models: {e}")

print("\n" + "=" * 60)

---
## 3. Phase 1: Data Pipeline

In [ ]:
#@title 3.1 Configure Pipeline { display-mode: "form" }
#@markdown Configure the data processing pipeline.

#@markdown ### Symbol Selection
symbols = "MES"  #@param {type: "string"}
#@markdown Comma-separated symbols (e.g., "MES,MGC")

#@markdown ### Label Horizons
horizons = "5,10,15,20"  #@param {type: "string"}
#@markdown Comma-separated horizons (bars ahead)

#@markdown ### Train/Val/Test Split
train_ratio = 0.70  #@param {type: "slider", min: 0.5, max: 0.8, step: 0.05}
val_ratio = 0.15  #@param {type: "slider", min: 0.1, max: 0.25, step: 0.05}

# Parse inputs
SYMBOLS = [s.strip().upper() for s in symbols.split(',')]
HORIZONS = [int(h.strip()) for h in horizons.split(',')]
TRAIN_RATIO = train_ratio
VAL_RATIO = val_ratio
TEST_RATIO = round(1.0 - train_ratio - val_ratio, 2)

print("Pipeline Configuration:")
print(f"  Symbols: {SYMBOLS}")
print(f"  Horizons: {HORIZONS}")
print(f"  Train/Val/Test: {TRAIN_RATIO}/{VAL_RATIO}/{TEST_RATIO}")

In [ ]:
#@title 3.2 Run Data Pipeline OR Use Existing Data { display-mode: "form" }
#@markdown Choose whether to run the full pipeline or use existing processed data.

data_source = "Use existing processed data"  #@param ["Run full pipeline (requires raw data)", "Use existing processed data"]

from pathlib import Path
import time

# CORRECT: Data from GitHub clone (not Google Drive)
splits_dir = Path('/content/research/data/splits/scaled')
train_file = splits_dir / "train_scaled.parquet"

if data_source == "Use existing processed data":
    if train_file.exists():
        import pandas as pd
        train_df = pd.read_parquet(train_file)
        val_df = pd.read_parquet(splits_dir / "val_scaled.parquet")
        test_df = pd.read_parquet(splits_dir / "test_scaled.parquet")
        
        print("Found existing processed data!")
        print(f"  Location: {splits_dir}")
        print(f"  Train: {len(train_df):,} samples")
        print(f"  Val: {len(val_df):,} samples")
        print(f"  Test: {len(test_df):,} samples")
        print("\nSkipping pipeline - proceeding to model training!")
    else:
        print("ERROR: Processed data not found!")
        print(f"  Expected: {splits_dir}/")
        print("\nMake sure the GitHub repo contains processed data files:")
        print("  - train_scaled.parquet")
        print("  - val_scaled.parquet")
        print("  - test_scaled.parquet")
else:
    raw_dir = Path('/content/research/data/raw')
    raw_files = list(raw_dir.glob("*.parquet")) + list(raw_dir.glob("*.csv")) if raw_dir.exists() else []
    
    if not raw_files:
        print("ERROR: No raw data files found!")
        print(f"  Expected: {raw_dir}/MES_1m.parquet or .csv")
    else:
        print("Running Phase 1 Data Pipeline...")
        print("=" * 60)
        start_time = time.time()
        
        try:
            from src.phase1.pipeline_config import PipelineConfig
            from src.pipeline.runner import PipelineRunner
            
            config = PipelineConfig(
                symbols=SYMBOLS,
                project_root=Path('/content/research'),
                label_horizons=HORIZONS,
                train_ratio=TRAIN_RATIO,
                val_ratio=VAL_RATIO,
                test_ratio=TEST_RATIO,
            )
            
            runner = PipelineRunner(config)
            success = runner.run()
            
            elapsed = time.time() - start_time
            print("\n" + "=" * 60)
            if success:
                print(f"Pipeline completed in {elapsed/60:.1f} minutes!")
            else:
                print("Pipeline failed. Check errors above.")
        except Exception as e:
            print(f"\nError: {e}")
            import traceback
            traceback.print_exc()

In [ ]:
#@title 3.3 Verify Processed Data { display-mode: "form" }
#@markdown Loads and displays the processed datasets.

import pandas as pd
from pathlib import Path

# CORRECT: Data from GitHub clone (not Google Drive)
splits_dir = Path('/content/research/data/splits/scaled')

print("Loading processed datasets...")
print("=" * 60)

try:
    train_df = pd.read_parquet(splits_dir / "train_scaled.parquet")
    val_df = pd.read_parquet(splits_dir / "val_scaled.parquet")
    test_df = pd.read_parquet(splits_dir / "test_scaled.parquet")
    
    print(f"\nDataset sizes:")
    print(f"  Train: {len(train_df):,} samples")
    print(f"  Val:   {len(val_df):,} samples")
    print(f"  Test:  {len(test_df):,} samples")
    print(f"  Total: {len(train_df) + len(val_df) + len(test_df):,} samples")
    
    feature_cols = [c for c in train_df.columns if not c.startswith(('label_', 'sample_weight', 'quality_score', 'datetime', 'symbol'))]
    label_cols = [c for c in train_df.columns if c.startswith('label_')]
    
    print(f"\nFeatures: {len(feature_cols)}")
    print(f"Labels: {label_cols}")
    
    print(f"\nLabel distribution (train):")
    for col in label_cols:
        dist = train_df[col].value_counts().sort_index()
        print(f"  {col}: Long={dist.get(1, 0):,} | Neutral={dist.get(0, 0):,} | Short={dist.get(-1, 0):,}")
    
    TRAIN_DF = train_df
    VAL_DF = val_df
    TEST_DF = test_df
    FEATURE_COLS = feature_cols
    
    print("\nData ready for model training!")
    
except FileNotFoundError:
    print("Processed data not found. Run Section 3.2 first.")

---
## 4. Phase 2: Model Training

In [ ]:
#@title 4.1 Training Mode Selection { display-mode: "form" }
#@markdown Choose your training mode and models.

training_mode = "Single Model"  #@param ["Single Model", "Multi-Model (Sequential)"]

#@markdown ---
#@markdown ### Single Model Options
single_model = "xgboost"  #@param ["xgboost", "lightgbm", "catboost", "random_forest", "logistic", "svm", "lstm", "gru", "tcn"]

#@markdown ---
#@markdown ### Multi-Model Options
train_boosting = True  #@param {type: "boolean"}
#@markdown XGBoost, LightGBM, CatBoost
train_classical = False  #@param {type: "boolean"}
#@markdown Random Forest, Logistic, SVM
train_neural = False  #@param {type: "boolean"}
#@markdown LSTM, GRU, TCN (requires GPU)

#@markdown ---
#@markdown ### Training Parameters
horizon = 20  #@param [5, 10, 15, 20]
sequence_length = 60  #@param {type: "slider", min: 30, max: 120, step: 10}

# Build model list
if training_mode == "Single Model":
    MODELS_TO_TRAIN = [single_model]
else:
    MODELS_TO_TRAIN = []
    if train_boosting:
        MODELS_TO_TRAIN.extend(['xgboost', 'lightgbm', 'catboost'])
    if train_classical:
        MODELS_TO_TRAIN.extend(['random_forest', 'logistic', 'svm'])
    if train_neural and GPU_AVAILABLE:
        MODELS_TO_TRAIN.extend(['lstm', 'gru', 'tcn'])
    elif train_neural and not GPU_AVAILABLE:
        print("WARNING: Neural models skipped (no GPU)")

HORIZON = horizon
SEQ_LEN = sequence_length

print(f"Training Mode: {training_mode}")
print(f"Models to train: {MODELS_TO_TRAIN}")
print(f"Horizon: H{HORIZON}")
if any(m in ['lstm', 'gru', 'tcn'] for m in MODELS_TO_TRAIN):
    print(f"Sequence length: {SEQ_LEN}")

In [ ]:
#@title 4.2 Train Models { display-mode: "form" }
#@markdown Execute model training based on your selections.

import time
from pathlib import Path

print("=" * 60)
print(" MODEL TRAINING")
print("=" * 60)

TRAINING_RESULTS = {}

try:
    from src.models import ModelRegistry, Trainer, TrainerConfig
    from src.phase1.stages.datasets.container import TimeSeriesDataContainer
    
    # CORRECT: Load data from GitHub clone (not Google Drive)
    print(f"\nLoading data for horizon H{HORIZON}...")
    container = TimeSeriesDataContainer.from_parquet_dir(
        path=Path('/content/research/data/splits/scaled'),
        horizon=HORIZON
    )
    print(f"  Train samples: {container.splits['train'].n_samples:,}")
    print(f"  Val samples: {container.splits['val'].n_samples:,}")
    print(f"  Features: {container.n_features}")
    
    for i, model_name in enumerate(MODELS_TO_TRAIN, 1):
        print(f"\n{'='*60}")
        print(f" [{i}/{len(MODELS_TO_TRAIN)}] Training: {model_name.upper()}")
        print("=" * 60)
        
        start_time = time.time()
        
        # Configure - save results to Google Drive
        if model_name in ['lstm', 'gru', 'tcn']:
            config = TrainerConfig(
                model_name=model_name,
                horizon=HORIZON,
                sequence_length=SEQ_LEN,
                batch_size=RECOMMENDED_BATCH_SIZE,
                max_epochs=50,
                early_stopping_patience=10,
                output_dir=Path('/content/drive/MyDrive/research/experiments/runs'),
                device="cuda" if GPU_AVAILABLE else "cpu",
                mixed_precision=MIXED_PRECISION,
            )
        else:
            config = TrainerConfig(
                model_name=model_name,
                horizon=HORIZON,
                output_dir=Path('/content/drive/MyDrive/research/experiments/runs'),
            )
        
        trainer = Trainer(config)
        results = trainer.run(container)
        
        elapsed = time.time() - start_time
        
        TRAINING_RESULTS[model_name] = {
            'metrics': results.get('evaluation_metrics', {}),
            'time': elapsed,
            'run_id': results.get('run_id', 'unknown'),
        }
        
        metrics = results.get('evaluation_metrics', {})
        print(f"\n  Results:")
        print(f"    Accuracy: {metrics.get('accuracy', 0):.2%}")
        print(f"    Macro F1: {metrics.get('macro_f1', 0):.4f}")
        print(f"    Time: {elapsed:.1f}s")
        
except Exception as e:
    print(f"\nError during training: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 60)
print(" TRAINING COMPLETE")
print("=" * 60)

In [ ]:
#@title 4.3 Compare Results { display-mode: "form" }
#@markdown Display comparison of all trained models.

import pandas as pd
import matplotlib.pyplot as plt

if TRAINING_RESULTS:
    print("Model Comparison")
    print("=" * 60)
    
    rows = []
    for model, data in TRAINING_RESULTS.items():
        metrics = data['metrics']
        rows.append({
            'Model': model,
            'Accuracy': metrics.get('accuracy', 0),
            'Macro F1': metrics.get('macro_f1', 0),
            'Weighted F1': metrics.get('weighted_f1', 0),
            'Time (s)': data['time'],
        })
    
    comparison_df = pd.DataFrame(rows)
    comparison_df = comparison_df.sort_values('Macro F1', ascending=False)
    print(comparison_df.to_string(index=False))
    
    if len(TRAINING_RESULTS) > 1:
        fig, axes = plt.subplots(1, 2, figsize=(12, 4))
        
        comparison_df_sorted = comparison_df.sort_values('Accuracy', ascending=True)
        axes[0].barh(comparison_df_sorted['Model'], comparison_df_sorted['Accuracy'])
        axes[0].set_xlabel('Accuracy')
        axes[0].set_title('Model Accuracy Comparison')
        axes[0].set_xlim(0, 1)
        
        comparison_df_sorted = comparison_df.sort_values('Time (s)', ascending=True)
        axes[1].barh(comparison_df_sorted['Model'], comparison_df_sorted['Time (s)'])
        axes[1].set_xlabel('Training Time (seconds)')
        axes[1].set_title('Training Time Comparison')
        
        plt.tight_layout()
        plt.show()
    
    best_model = comparison_df.iloc[0]['Model']
    print(f"\nBest model: {best_model}")
else:
    print("No training results yet. Run Section 4.2 first.")

---
## 5. Phase 3: Cross-Validation (Optional)

In [ ]:
#@title 5.1 Run Cross-Validation { display-mode: "form" }
#@markdown Run purged k-fold cross-validation for robust evaluation.

run_cv = False  #@param {type: "boolean"}
cv_model = "xgboost"  #@param ["xgboost", "lightgbm", "catboost", "random_forest"]
n_splits = 5  #@param {type: "slider", min: 3, max: 10, step: 1}

if run_cv:
    print(f"Running {n_splits}-fold cross-validation for {cv_model}...")
    print("=" * 60)
    
    # CORRECT: Data from GitHub clone, results to Google Drive
    !python scripts/run_cv.py \
        --models {cv_model} \
        --horizons {HORIZON} \
        --n-splits {n_splits} \
        --data-dir /content/research/data/splits/scaled \
        --output-dir /content/drive/MyDrive/research/results/cv
else:
    print("Cross-validation skipped. Enable run_cv to run.")

---
## 6. Phase 4: Ensemble Training (Optional)

In [ ]:
#@title 6.1 Train Ensemble { display-mode: "form" }
#@markdown Combine multiple models into an ensemble.

train_ensemble = False  #@param {type: "boolean"}
ensemble_type = "voting"  #@param ["voting", "stacking", "blending"]
base_models = "xgboost,lightgbm,catboost"  #@param {type: "string"}

if train_ensemble:
    print(f"Training {ensemble_type} ensemble...")
    print(f"Base models: {base_models}")
    print("=" * 60)
    
    try:
        from src.models import ModelRegistry, Trainer, TrainerConfig
        from src.phase1.stages.datasets.container import TimeSeriesDataContainer
        
        # CORRECT: Load data from GitHub clone (not Google Drive)
        container = TimeSeriesDataContainer.from_parquet_dir(
            path=Path('/content/research/data/splits/scaled'),
            horizon=HORIZON
        )
        
        # Save results to Google Drive
        config = TrainerConfig(
            model_name=ensemble_type,
            horizon=HORIZON,
            output_dir=Path('/content/drive/MyDrive/research/experiments/runs'),
            model_config={
                "base_model_names": [m.strip() for m in base_models.split(',')],
            }
        )
        
        trainer = Trainer(config)
        results = trainer.run(container)
        
        metrics = results.get('evaluation_metrics', {})
        print(f"\nEnsemble Results:")
        print(f"  Accuracy: {metrics.get('accuracy', 0):.2%}")
        print(f"  Macro F1: {metrics.get('macro_f1', 0):.4f}")
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print("Ensemble training skipped. Enable train_ensemble to run.")

---
## 7. Save Results & Next Steps

In [ ]:
#@title 7.1 Summary & Saved Artifacts { display-mode: "form" }
#@markdown Display summary and location of all saved files.

from pathlib import Path

print("=" * 60)
print(" SESSION SUMMARY")
print("=" * 60)

# Data summary
print("\n DATA (from GitHub clone):")
splits_dir = Path('/content/research/data/splits/scaled')
if splits_dir.exists():
    for f in splits_dir.glob("*.parquet"):
        size_mb = f.stat().st_size / 1e6
        print(f"  {f.name}: {size_mb:.1f} MB")

# Training results
print("\n TRAINED MODELS (saved to Google Drive):")
experiments_dir = Path('/content/drive/MyDrive/research/experiments/runs')
if experiments_dir.exists():
    runs = list(experiments_dir.iterdir())
    for run_dir in sorted(runs)[-5:]:
        if run_dir.is_dir():
            print(f"  {run_dir.name}")

# Next steps
print("\n NEXT STEPS:")
print("  1. Review model metrics in Google Drive: experiments/runs/")
print("  2. Try different model configurations")
print("  3. Run cross-validation for robust evaluation")
print("  4. Train ensemble for best performance")
print("  5. Export best model for production")

print("\n" + "=" * 60)
print(" Data loaded from: /content/research")
print(" Results saved to: /content/drive/MyDrive/research")
print("=" * 60)

---
## Appendix: Quick Commands

```bash
# Train single model
!python scripts/train_model.py --model xgboost --horizon 20

# Train neural model
!python scripts/train_model.py --model lstm --horizon 20 --seq-len 60

# Run cross-validation
!python scripts/run_cv.py --models xgboost,lightgbm --horizons 20 --n-splits 5

# Train ensemble
!python scripts/train_model.py --model voting --horizon 20

# List all models
!python scripts/train_model.py --list-models
```